In [141]:
import rasterio
from rasterio.transform import rowcol
from pyproj import Transformer

In [142]:
hennepin = rasterio.open("hennepin.tif")
green = rasterio.open("green.tif")

In [143]:
h_to_wgs84 = Transformer.from_crs(hennepin.crs, "EPSG:4326", always_xy=True)
h_from_wgs84 = Transformer.from_crs("EPSG:4326", hennepin.crs, always_xy=True)

g_to_wgs84 = Transformer.from_crs(green.crs, "EPSG:4326", always_xy=True)
g_from_wgs84 = Transformer.from_crs("EPSG:4326", green.crs, always_xy=True)

In [144]:
lat, lon = 44.893108, -93.235012

x, y = h_from_wgs84.transform(lon, lat)
gx, gy = g_from_wgs84.transform(lon, lat)

row, col = rowcol(hennepin.transform, x, y)
grow, gcol = rowcol(green.transform, gx, gy)

In [145]:
h_band = hennepin.read(1)
g_band = green.read(1)

First we define the bounding box, which is the same bounding box of the green space band.

Now we write some code that gets a random acre inside the bounding box.

In [146]:
import numpy as np

ACRE_EDGE_METERS = 63.6149353533
DEGREES_PER_METER = 8.983e-6

lat = np.random.uniform(
    green.bounds.bottom + ACRE_EDGE_METERS * DEGREES_PER_METER, 
    green.bounds.top
)

lon = np.random.uniform(
    green.bounds.left, 
    green.bounds.right - ACRE_EDGE_METERS * DEGREES_PER_METER
)

lat, lon

(44.83348992665804, -93.12937472390509)

In [171]:
# Get the indices for everything

# Top left for green
x, y = g_from_wgs84.transform(lon, lat)
g_row_min, g_col_min = rowcol(green.transform, x, y)

# Bottom right for green
x, y = g_from_wgs84.transform(lon + ACRE_EDGE_METERS * DEGREES_PER_METER, lat - ACRE_EDGE_METERS * DEGREES_PER_METER)
g_row_max, g_col_max = rowcol(green.transform, x, y)

# Top left for hennepin heat map
x, y = h_from_wgs84.transform(lon, lat)
h_row_min, h_col_min = rowcol(hennepin.transform, x, y)

# Bottom right for hennepin heat map
x, y = h_from_wgs84.transform(lon + ACRE_EDGE_METERS * DEGREES_PER_METER, lat - ACRE_EDGE_METERS * DEGREES_PER_METER)
h_row_max, h_col_max = rowcol(hennepin.transform, x, y)

In [174]:
from rasterio.windows import Window
green_window = Window(col_off=g_col_min, row_off=g_row_min, width=g_col_max-g_col_min+1, height=g_row_max-g_row_min+1)
heat_window = Window(col_off=h_col_min, row_off=h_row_min, width=h_col_max-h_col_min+1, height=h_row_max-h_row_min+1)


In [177]:
green_data = green.read(1, window=green_window)
heat_data = hennepin.read(1, window=heat_window)

In [179]:
green_data.shape

(8, 7)

In [180]:
heat_data.shape

(8, 5)

In [ ]:
green_data.mean(), heat_data.mean()

(<function ndarray.mean>, <function ndarray.mean>)